In [1]:
import sys
sys.path.append("../src/")
from data import train_valid_testloader_from_pc, LogTransform, NonTransform, StandardScaler, MinMaxScaler, PowerTransformer
from model import AEDist
import numpy as np
import torch
import phate
from heatgeo.embedding import HeatGeo
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from scipy.spatial import procrustes
import scanpy as sc
import scprep
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
import hydra
import os
from omegaconf import DictConfig, OmegaConf
import wandb
from hydra import initialize, compose
import pickle
import pandas as pd
from metrics import mAP, computeKNNmAP

In [2]:
data_all = np.load('../data/eb_subset.npz')#, data=data, phate=Y_phate, dist=X_dist, colors=colors.values)

In [3]:
data = data_all['data']
X_dist = data_all['dist']
Y_phate = data_all['phate']
colors = data_all['colors']

In [4]:
cfg = OmegaConf.load(f'../src/wandb/run-20240116_211824-ydvuqitz/files/config.yaml')

In [5]:
activation_dict = {
    'relu': torch.nn.ReLU(),
    'leaky_relu': torch.nn.LeakyReLU(),
    'sigmoid': torch.nn.Sigmoid()
}

activation_fn = activation_dict[cfg.model.value['activation']]

In [6]:
preprocessor_dict = {
    'standard': StandardScaler(),
    'minmax': MinMaxScaler(),
    'power': PowerTransformer(),
    'log': LogTransform(),
    'none': NonTransform()
}
pp = preprocessor_dict[cfg.data.value['preprocess']]

In [7]:
ckpt_path=f'../src/wandb/run-20240116_211824-ydvuqitz/files/epoch=49-step=96000.ckpt'
activation_dict = {
    'relu': torch.nn.ReLU(),
    'leaky_relu': torch.nn.LeakyReLU(),
    'sigmoid': torch.nn.Sigmoid()
}
model = AEDist.load_from_checkpoint(
    checkpoint_path=ckpt_path,
    dim=data.shape[1],
    emb_dim=Y_phate.shape[1],
    layer_widths=cfg.model.value['layer_widths'],
    activation_fn=activation_fn,
    dist_reconstr_weights=cfg['model.dist_reconstr_weights'].value,
    pp=pp,
    lr=cfg.model.value['lr'],
    dist_recon_topk_coords=cfg.model.value['dist_recon_topk_coords'],
)

In [9]:
phate_D = X_dist
shapes = phate_D.shape
phate_D = pp.fit_transform(phate_D.reshape(-1,1)).reshape(shapes)

In [10]:
X = data

In [71]:
trainloader, valloader, testloader = train_valid_testloader_from_pc(
    X, # <---- Pointcloud
    phate_D, # <---- Distance matrix to match
    batch_size=cfg.training.value['batch_size'],
    train_test_split=cfg.training.value['train_test_split'],
    train_valid_split=cfg.training.value['train_valid_split'],
    shuffle=cfg.training.value['shuffle'],
    seed=cfg.training.value['seed'],)

In [72]:
# torch.manual_seed(0)
# position_sum = 0
# # Position you want to sum (e.g., row = 1, column = 2)
# row_idx = 1
# col_idx = 2

# # Iterate over the DataLoader
# for batch in valloader:
#     batch = batch['x']
#     # Assuming batch is a tensor
#     # Update the sum with the value at the specified position in each tensor
#     position_sum += batch[row_idx, col_idx].sum()
# position_sum

tensor(-19.5657)

In [ ]:
with torch.no_grad():
    model.eval()
    for batch in testloader:
        x = batch['x']
        x_hat, emb_z = model(x)